In [8]:
df_start = pd.read_csv('/content/df_9_after_cleansing.csv')
df_start.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2973 entries, 0 to 2972
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        2973 non-null   int64  
 1   Date              2973 non-null   object 
 2   Journal           2971 non-null   object 
 3   Title             2973 non-null   object 
 4   Name              2973 non-null   object 
 5   wavelength_added  2973 non-null   float64
 6   RI                2973 non-null   float64
 7   DOI_alt           2973 non-null   object 
 8   SMILES            2973 non-null   object 
 9   Method_ord        2973 non-null   int64  
 10  Publisher_ord     2973 non-null   int64  
dtypes: float64(2), int64(3), object(6)
memory usage: 255.6+ KB


# RDkit descriptors

In [4]:
!pip install rdkit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from rdkit.Chem import Descriptors, MolFromSmiles

In [ ]:
#convert smiles to mol object
df_start['mol'] = df_start['SMILES'].apply(lambda x: MolFromSmiles(x))

In [ ]:
Descriptors.descList

In [ ]:
for i,j in tqdm(Descriptors.descList):
    df_start[i] = df_start['mol'].map(j)

# PADELpy descriptors

In [2]:
!pip install padelpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 48.8 MB/s eta 0:00:00


In [3]:
from padelpy import from_smiles

In [4]:
padel_list =list(from_smiles('CC').keys())

In [9]:
#search descriptors only for unique smiles
df_short = df_start.groupby(['SMILES'], as_index = False).mean()
df_short

,SMILES,Unnamed: 0,wavelength_added,RI,Method_ord,Publisher_ord
0,*C,1556.00,589.0,1.3749,0.00,0.0
1,*CC,914.00,589.0,1.4800,0.00,0.0
2,B#N,3814.00,560.0,1.4686,0.00,0.0
3,B#[Sb],2791.00,589.0,3.3500,0.00,0.0
4,B#[V],139.00,589.0,1.3420,0.00,0.0
...,...,...,...,...,...,...
860,[Tm],650.50,589.0,2.4105,0.50,0.0
861,[Xe],1537.00,589.0,1.3400,1.00,0.0
862,[Zn+2].[I-].[I-],4944.00,589.0,1.6200,2.00,2.0
863,[Zn]=[Se],2815.20,589.0,2.4526,1.60,0.0


In [10]:
#5 hours
df_padel = pd.DataFrame(columns = padel_list)
for i, _ in tqdm(df_short.iterrows()):
  try:
    df_padel = pd.concat([df_padel, pd.DataFrame(from_smiles(df_short['SMILES'][i]), index = [df_short['SMILES'][i]])])
  except:
    df_padel = pd.concat([df_padel, pd.DataFrame(np.nan, columns = padel_list, index = [df_short['SMILES'][i]])])

865it [5:07:33, 21.33s/it]


In [15]:
df_padel.describe()

,nAcid,ALogP,ALogp2,AMR,apol,naAromAtom,nAromBond,nAtom,nHeavyAtom,nH,...,P1s,P2s,E1s,E2s,E3s,Ts,As,Vs,Ks,Ds
count,789,789,789,789,789,789,789,789,789,789,...,789,789,789,789,789,789,789,789,789,789
unique,5,541,541,540,703,19,16,88,55,58,...,420,421,423,423,412,423,423,423,419,418
top,0,,,,18.561515999999997,0,0,5,5,0,...,,,,,,,,,,
freq,703,178,178,178,7,651,651,76,98,230,...,366,366,366,366,371,366,366,366,366,371


In [12]:
df_padel.to_csv('dict_for_padel_descriptors.csv')